In [ ]:
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import seaborn as sns

from eftqpe.physical_costing import (
    fermi_hubbard_ftqc_physical_cost,
)
from eftqpe.utils import make_decreasing_function

plt.style.use("figstyle.mplstyle")

## Physical costs for circuit-division SinQPE

In [ ]:
save_file = 'data/physical_costs/fermi_hubbard_multicircuit_long_run.csv'

cost_df = pd.read_csv(save_file)
delta_e = cost_df['delta_e'][0]
side_list = np.sort(cost_df['side'].unique())
n_factories_list = np.sort(cost_df['n_factories'].unique())
gamma_list = np.sort(cost_df['gamma'].unique())

In [ ]:
cost_df

## Fermi-Hubbard costing plots

In [ ]:
colors = sns.color_palette("flare", len(side_list))


fig = plt.figure(
    # layout="constrained",
    figsize=(7, 3.5)
)
fig1, fig2 = fig.subfigures(1, 2, wspace=0.0, width_ratios=[1.0, 1.])

ax = fig1.subplots(2, 2, sharex=True, sharey='row', gridspec_kw={'hspace': 0, 'wspace':0})
ax2 = fig2.subplots(1, 1)

for k, n_factories in enumerate(n_factories_list):
    for j, side in enumerate(side_list):
        sub_df = cost_df.query(f'side == {side} & n_factories == {n_factories}')
        ax[0, k].plot(sub_df.gamma, sub_df.t_tot_hr, '-', color = colors[j])
        ax[0, k].plot(sub_df.gamma, sub_df.t_max_hr, '--', color = colors[j])
        ax[1, k].plot(sub_df.gamma, sub_df.footprint, '.', color = colors[j],
                      label=f'$L = {side}$')
        
    ax[0, k].set_title(f'{n_factories} CCZ factor'+('y' if n_factories == 1 else 'ies'))

ax[0,0].set_ylabel('runtime in hours')
ax[0,0].legend([plt.Line2D([0], [0], color='grey', linestyle=ls) for ls in ['-', '--']],
              [r'$\mathcal{T}_{\mathrm{tot}}$', r'$\mathcal{T}_{\mathrm{max}}$'], ncol=1)

ax[1,0].set_ylabel(r'\# physical qubits')
ax[1,1].legend(ncol=1, loc='lower left', handlelength=0.3)

fig1.supxlabel(r'$\gamma$, error rate per walk step', x=0.5, horizontalalignment='center', fontsize="medium")

ax[0,0].set_yscale('log')
ax[1,0].set_yscale('log')
ax[1,0].set_xscale('log')

markers = ["+-", "x:"]

for i, n_factories in enumerate(n_factories_list):
    for j, side in enumerate(side_list):
        sub_df = cost_df.query(f"side == {side} & n_factories == {n_factories}")
        footprints, runtimes = make_decreasing_function(sub_df.footprint, sub_df.t_tot_hr)

        ax2.plot(
            footprints,
            runtimes,
            markers[i],
            color=colors[j],
            label=f"$L = {side}$",
        )

handles = [
    plt.Line2D([0], [0], color="grey", marker=marker[0], linestyle=marker[1]) for marker in markers
] + [plt.Line2D([0], [0], color=c) for c in colors]

labels = [f"{n_factories} {'factories' if n_factories > 1 else 'factory'}" for n_factories in n_factories_list] + [
    rf"$L = {side}$" for side in side_list
]
ax2.legend(handles, labels, ncol=2, loc='upper left')

ax2.axhline(24, color="black", linestyle="dashed")
ax2.text(1.01e5, 1.3 * 24, "1 day", va="bottom")
ax2.axhline(24 * 30, color="black", linestyle="dashed")
ax2.text(1.5e5, 1.3 * 24 * 30, "1 month", va="bottom")

# ax2.set_xlabel(r"\# physical qubits")
fig2.supxlabel(r"\# physical qubits", x=0.5, fontsize="medium")

ax2.grid(axis="x", which="minor")
ax2.set_ylabel("runtime in hours")
ax2.set_yscale("log")
ax2.set_xscale("log")

ax2.set_title(" ")

plt.savefig("fermi_hubbard_costing.pdf", bbox_inches='tight')

## Comparison to standard FTQC algorithm

In [ ]:
delta_e = 1e-2
side_list = [2, 5, 10, 20]
n_factories_list = [1, 2]
error_budget_list = [1e-2, 1e-3]

ftqc_costs = []

for side, n_factories, error_budget in tqdm(
    list(product(side_list, n_factories_list, error_budget_list))
):
    cost_dict = fermi_hubbard_ftqc_physical_cost(
        side=side, error_budget=error_budget, delta_e=delta_e, n_factories=n_factories,
    )
    # remove unnecessary keys
    cost_dict.pop("physical_cost", None)
    cost_dict.pop("factory", None)
    cost_dict.pop("data_block", None)
    # add input parameters
    cost_dict.update(
        {"side": side, "n_factories": n_factories, "error_budget": error_budget}
    )
    ftqc_costs.append(cost_dict)

ftqc_cost_df = pd.DataFrame(ftqc_costs)

In [ ]:
ftqc_cost_df

# THC Costing

In [ ]:
def get_npz_files(out):
    import glob
    npzs = glob.glob(f"{out}/*.npz")
    nacts = [int(npz.split("_")[-4]) for npz in npzs]
    npzs = [x for _, x in sorted(zip(nacts, npzs))]
    return npzs

systems = get_npz_files("data/thc")

cas = {}
for npz in systems:
    mol = npz.split("/")[-1].split("_thc")[0]
    nact = int(npz.split("_")[-4])
    nalpha = int(npz.split("_")[-2])
    nbeta = int(npz.split("_")[-3])
    print(mol, nact, nalpha, nbeta)
    cas[mol] = f"({nalpha+nbeta},{nact})"

In [ ]:
import glob
h5files = glob.glob("data/thc/resources/*.h5")
dfs = [pd.read_hdf(f, key="df") for f in h5files]
df = pd.concat(dfs, ignore_index=True)

dfs = [pd.read_hdf(f, key="df_ftqc") for f in h5files]
df_ftqc = pd.concat(dfs, ignore_index=True)

label_mol = {
    "n2": r"N$_2$",
    "h2o": r"H$_2$O",
    "naphthalene": "Naphthalene",
    "anthracene": "Anthracene",
    "a-S12-large": "Co(salophen)",
}

## Fermi-Hubbard/THC Runtime vs. Physical Cost

In [ ]:
fig = plt.figure(figsize=(3.4 * 2, 2.5))
ax1, ax2 = fig.subplots(1, 2, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0.02})

colors = sns.color_palette("flare", len(side_list))
markers_ftqc = ["^", "v"]
n_factories = 1

for j, side in enumerate(side_list):
    sub_df = cost_df.query(f"side == {side} & n_factories == {n_factories}")
    footprints, runtimes = make_decreasing_function(sub_df.footprint, sub_df.t_tot_hr)

    ax1.plot(
        footprints,
        runtimes,
        '.-',
        color=colors[j],
        label=f"$L = {side}$",
    )

    for i, error_budget in enumerate(error_budget_list):
        sub_df = ftqc_cost_df.query(f"side == {side} & n_factories == {n_factories} & error_budget == {error_budget}")
        ax1.plot(
            sub_df.footprint,
            sub_df.runtime_hr,
            markers_ftqc[i],
            color=colors[j],
            alpha=0.8
        )


handles = (
    [plt.Line2D([0], [0], color=c) for c in colors]
)
labels = (
    [rf"$L = {side}$" for side in side_list]
)

ax1.axhline(24, color="black", linestyle="dashed")
ax1.text(1.01e5, 1.3 * 24, "1 day", va="bottom")
ax1.axhline(24 * 30, color="black", linestyle="dashed")
ax1.text(1.5e5, 1.3 * 24 * 30, "1 month", va="bottom")


### THC
mol_list = [
    'n2', 'h2o', 'naphthalene', 'anthracene', 'a-S12-large',
]

colors = sns.color_palette("tab10", len(mol_list))
# colors = sns.color_palette("colorblind", len(mol_list))
for j, mol in enumerate(mol_list):
    sub_df = df.query(f"molecule == '{mol}' & n_factories == {n_factories}")
    footprints, runtimes = make_decreasing_function(sub_df.footprint, sub_df.t_tot_hr)

    ax2.plot(
        footprints,
        runtimes,
        '.-',
        color=colors[j],
    )

    for i, error_budget in enumerate(error_budget_list):
        sub_df = df_ftqc.query(f"molecule == '{mol}' & n_factories == {n_factories} & error_budget == {error_budget}")
        ax2.plot(
            sub_df.footprint,
            sub_df.runtime_hr,
            markers_ftqc[i],
            color=colors[j],
            alpha=0.8,
        )


handles += [plt.Line2D([0], [0], color=c) for c in colors]
labels += [label_mol[mol] + f"/{cas[mol]}" for mol in mol_list]


handles += [
    plt.Line2D([0], [0], color="grey", marker=marker, linestyle="none")
    for marker in markers_ftqc
]
labels += [rf"{error_budget*100}\% fail" for error_budget in error_budget_list]

ax2.axhline(24, color="black", linestyle="dashed")
ax2.axhline(24 * 30, color="black", linestyle="dashed")


for ax in [ax1, ax2]:
    ax.set_xlabel(r"\# physical qubits")
    ax.grid(axis="x", which="minor")
    ax.set_yscale("log")
    ax.set_xscale("log")

ax1.set_ylabel("runtime in hours")
ax1.set_xlim(7e4, 4e6)
ax2.set_xlim(1e5, 2e6)

yticks = np.arange(-1, 7)
yticks_label = [rf'$10^{num}$' if num % 2 == 0 else ' ' for num in yticks]
ax1.set_yticks(np.power(10., yticks))
ax1.set_yticklabels(yticks_label)


import matplotlib.text as mtext

class LegendTitle(object):
    def __init__(self, text_props=None):
        self.text_props = text_props or {}
        super(LegendTitle, self).__init__()

    def legend_artist(self, legend, orig_handle, fontsize, handlebox):
        x0, y0 = handlebox.xdescent, handlebox.ydescent
        title = mtext.Text(x0, y0, orig_handle, usetex=True, **self.text_props)
        handlebox.add_artist(title)
        return title

handles.insert(0, "Fermi-Hubbard")
labels.insert(0, "")

handles.insert(5, "Electronic Structure")
labels.insert(5, "")

handles.insert(11, "Standard FTQC Algorithm")
labels.insert(11, "")
fig.legend(handles, labels, loc='upper left', ncol=1,
           bbox_to_anchor=(0.9, 0.9),
           fontsize=7, handler_map={str: LegendTitle({'fontsize': 7})}
           )

ax1.set_title("Fermi-Hubbard")
ax2.set_title("Electronic Structure")
plt.savefig("runtime_fh_thc.pdf", bbox_inches='tight')

In [ ]:
fig = plt.figure(
    figsize=(7, 3.5)
)
colors = sns.color_palette("tab10", len(mol_list))
fig1, fig2 = fig.subfigures(1, 2, wspace=0.0, width_ratios=[1.0, 1.])

ax = fig1.subplots(2, 2, sharex=True, sharey='row', gridspec_kw={'hspace': 0, 'wspace':0})
ax2 = fig2.subplots(1, 1)
markers = ["+-", "x:"]

for k, n_factories in enumerate(n_factories_list):
    for j, mol in enumerate(mol_list):
        sub_df = df.query(f'molecule == "{mol}" & n_factories == {n_factories}')
        ax[0, k].plot(sub_df.gamma, sub_df.t_tot_hr, '-', color = colors[j])
        ax[0, k].plot(sub_df.gamma, sub_df.t_max_hr, '--', color = colors[j])
        ax[1, k].plot(sub_df.gamma, sub_df.footprint, '.', color = colors[j],
                      label=label_mol[mol])
        
        try:
            footprints, runtimes = make_decreasing_function(sub_df.footprint, sub_df.t_tot_hr)
        except Exception:
            continue
        ax2.plot(
            footprints,
            runtimes,
            markers[k],
            color=colors[j],
            label=label_mol[mol],
        )
        
    ax[0, k].set_title(f'{n_factories} CCZ factor'+('y' if n_factories == 1 else 'ies'))

ax[0,0].set_ylabel('runtime in hours')
ax[0,0].legend([plt.Line2D([0], [0], color='grey', linestyle=ls) for ls in ['-', '--']],
              [r'$\mathcal{T}_{\mathrm{tot}}$', r'$\mathcal{T}_{\mathrm{max}}$'], ncol=1)

ax[1,0].set_ylabel(r'\# physical qubits')
fig1.supxlabel(r'$\gamma$, error rate per walk step', x=0.5, horizontalalignment='center', fontsize="medium")

ax[0,0].set_yscale('log')
ax[1,0].set_yscale('log')
ax[1,0].set_xscale('log')

ax[1,0].set_ylim(1e5, 3e6)


handles = [plt.Line2D([0], [0], color=c) for c in colors] + [
    plt.Line2D([0], [0], color="grey", marker=marker[0], linestyle=marker[1]) for marker in markers
]

factory_label = {
    1: "factory",
}

labels = [label_mol[mol] for mol in mol_list] + [f"{n_factories} {factory_label.get(n_factories, "factories")}" for n_factories in n_factories_list]

ax2.axhline(24, color="black", linestyle="dashed")
ax2.text(1.7e6, 1.3 * 24, "1 day", va="bottom")
ax2.axhline(24 * 30, color="black", linestyle="dashed")
ax2.text(1.7e6, 1.3 * 24 * 30, "1 month", va="bottom")

ax2.axhline(24 * 365, color="black", linestyle="dashed")
ax2.text(1.7e6, 1.3 * 24 * 365, "1 year", va="bottom")

ax2.set_xlabel(r"\# physical qubits")
ax2.grid(axis="x", which="minor")
ax2.set_ylabel("runtime in hours")
ax2.set_yscale("log")
ax2.set_xscale("log")
ax2.set_xlim(1e5, 3e6)

fig.legend(handles, labels, ncol=3, loc="lower center", bbox_to_anchor=(0.5, 0.95))
plt.savefig("thc_costing.pdf", bbox_inches='tight')